# Directly compare the parameter errors from two Fisher programs

In [156]:
import crosspower as cp #branch quickCl
import FisherCl as fcl
#import camb

# set plotting to be in the notebook instead of its own qt window
%matplotlib inline

# Load my Fisher Matrix

In [157]:
# load saved Fobj

import FisherCl as fcl
import crosspower as cp
import pickle

filehandler = open('Fisher_4_4W.obj','r') # accuracyBoost=3,doNorm=True,lmax=2000, nz=3e5; nonlinear=True,
#neutrino_hierarchy = 'degenerate'; from FisherCl branch quickCl

#filehandler = open('Fisher_4_4U.obj','r') # accuracyBoost=3,doNorm=True,lmax=1000, epsrel = 1.49e-6; nonlinear=True,
#dw=0.05, neutrino_hierarchy = 'degenerate', from FisherCl branch master
#filehandler = open('Fisher_4_4T.obj','r') # accuracyBoost=3,doNorm=True,lmax=1000, epsrel = 1.49e-6; nonlinear=False,
#dw=0.05, neutrino_hierarchy = 'degenerate', from FisherCl branch master

Fobj = pickle.load(filehandler) 
lmin = Fobj.lmin
lmax = Fobj.lmax
nMaps = Fobj.nBins + 1
nCosParams = Fobj.nParams - Fobj.nBins
ells = Fobj.ells
obsNames = Fobj.obsNames
paramList = Fobj.paramList

print Fobj.obsList
print Fobj.obsNames
print Fobj.paramList
print Fobj.nonlinear
print Fobj.neutrino_hierarchy
print Fobj.binAs

[  0.   1.   2.   3.   4.   6.   7.   8.   9.  12.  13.  14.  18.  19.  24.]
['k,k', 'k,g1', 'k,g2', 'k,g3', 'k,g4', 'g1,g1', 'g1,g2', 'g1,g3', 'g1,g4', 'g2,g2', 'g2,g3', 'g2,g4', 'g3,g3', 'g3,g4', 'g4,g4']
['ombh2', 'omch2', 'cosmomc_theta', 'As', 'ns', 'tau', 'mnu', 'w', 'bin1', 'bin2', 'bin3', 'bin4']
True
degenerate
[ 1.  1.  1.  1.]


In [158]:
# define fsky factor
fsky =  0.5
print 'fsky =',fsky
print 'fsky^(-1/2) =',1/np.sqrt(fsky)

#Fij2 *= fsky
#sigmas2 = np.sqrt(np.diag(np.linalg.inv(Fij2)))

fsky = 0.5
fsky^(-1/2) = 1.41421356237


In [159]:
# does not include lmin = 50
# get my Fij
#FijZ = Fobj.Fij * fsky

# Load B's Fisher Matrix to have control over which parameters are fixed

In [160]:
# load some of Byeonghee's stuff
v3 = False
if v3:
    ByeDir = 'Byeonghee_dat3/'

    FijB = np.loadtxt(ByeDir+'Fisher_12params.dat')
    #Fij_B = (Y.T).reshape(12, 12)
    #print Fij_B-Fij_B.transpose()
else: #v4
    ByeDir = 'Byeonghee_dat4/'

    FijB = np.loadtxt(ByeDir+'Fullfisher_12params.dat')
    print FijB.shape
    
    
# message from B: 
# The parameters are ordered in this way: [b1, b2, b3, b4, cosmomc_theta, ombh2, omch2, ns, As, mnu, w, tau]
paramListB = ['b1', 'b2', 'b3', 'b4', 'cosmomc_theta', 'ombh2', 'omch2', 'ns', 'As', 'mnu', 'w', 'tau']
print paramListB

# my ordering:
paramList = Fobj.paramList
print paramList

(12, 12)
['b1', 'b2', 'b3', 'b4', 'cosmomc_theta', 'ombh2', 'omch2', 'ns', 'As', 'mnu', 'w', 'tau']
['ombh2', 'omch2', 'cosmomc_theta', 'As', 'ns', 'tau', 'mnu', 'w', 'bin1', 'bin2', 'bin3', 'bin4']


In [161]:
# define a permutation to map B's params to mine
perm = [5,6,4,8,7,11,9,10,0,1,2,3]
print paramList
print [paramListB[p] for p in perm]

['ombh2', 'omch2', 'cosmomc_theta', 'As', 'ns', 'tau', 'mnu', 'w', 'bin1', 'bin2', 'bin3', 'bin4']
['ombh2', 'omch2', 'cosmomc_theta', 'As', 'ns', 'tau', 'mnu', 'w', 'b1', 'b2', 'b3', 'b4']


# Function for fixing parameters by removal from Fisher matrix

In [162]:
# define row,col deleting function
def fixParam(Fij,pNum):
    return np.delete(np.delete(Fij,pNum,0),pNum,1)

# Function for array permutation

In [163]:
# define array permutating function

def arrayPermute(myArray,perm):
    """
    Purpose:
        Permute the ordering of the rows and columns of a square array. 
        Permutation will be done on the first two axes of input array.
    Inputs:
        myArray: an nxn square numpy array, possibly with more than 2 dimensions.
        perm: an n-length array, contaning a permutation of the numbers 0 to n-1
    Returns:
        A permutted numpy array of the same size and shape as input myArray
    """
    # get transpose permutator
    axesT = np.arange(myArray.shape.__len__())
    axesT[0],axesT[1] = axesT[1],axesT[0]
    #print axesT
    
    tempArray = np.array([myArray[p] for p in perm])
    tempArrayT = tempArray.transpose(axesT)  # switches axes 0,1 -> 1,0
    returnArrayT = np.array([tempArrayT[p] for p in perm])
    returnArray = returnArrayT.transpose(axesT)
    return returnArray


In [164]:
# permute B's Fisher matrix
FijBp = arrayPermute(FijB,perm)

In [165]:
# scale B's As values to match mine
AsIndex = 3
FijBp[AsIndex,:] *= 1e9 
FijBp[:,AsIndex] *= 1e9 

# Function to get sigmas

In [166]:
def getSigmas(Fij):
    return np.sqrt(np.diag(np.linalg.inv(Fij)))

# Calculate sigmas with various parameters held fixed

In [167]:
# get my Fij including lmin=50
lmin = 50
lmax = 1000
#lmin = 2
#lmax = 2000

verbose = False
FijZ = Fobj.makeFisher(lmin,lmax,verbose=verbose)

# apply fsky
FijZ *= fsky

# get sigmas for nothing held fixed
sigmasZ = getSigmas(FijZ)
sigmasB = getSigmas(FijBp)


In [168]:
# convert to sigma(b)/b
# actually, we're not using this.

#doSigBoverB = True
doSigBoverB = False

if doSigBoverB:
    binBs = Fobj.binBs
    print binBs

    sigmasZ[-4:]/=binBs
    print sigmasZ
    

In [169]:
# excise w, tau and remake sigmas

# remove tau: row,column = 5
toDelete = 5
paramList2 = np.delete(paramList,toDelete)
FijZ2 = fixParam(FijZ,toDelete)
FijB2 = fixParam(FijBp,toDelete)
# remove w: row,column = 6 (after deleting tau)
toDelete = 6
paramList3 = np.delete(paramList2,toDelete)
FijZ3 = fixParam(FijZ2,toDelete)
FijB3 = fixParam(FijB2,toDelete)

# get sigmas for tau, w held fixed
sigmasZ3 = getSigmas(FijZ3)
sigmasB3 = getSigmas(FijB3)

print paramList3

['ombh2' 'omch2' 'cosmomc_theta' 'As' 'ns' 'mnu' 'bin1' 'bin2' 'bin3'
 'bin4']


In [170]:
# excise mnu and remake sigmas
toDelete = 5
paramList4 = np.delete(paramList3,toDelete)
FijZ4 = fixParam(FijZ3,toDelete)
FijB4 = fixParam(FijB3,toDelete)

# get sigmas for tau, w, mnu held fixed
sigmasZ4 = getSigmas(FijZ4)
sigmasB4 = getSigmas(FijB4)

print paramList4

['ombh2' 'omch2' 'cosmomc_theta' 'As' 'ns' 'bin1' 'bin2' 'bin3' 'bin4']


In [171]:
# excise biases and remake sigmas
paramList5 = paramList4[:-4]
FijZ5 = FijZ4[:-4,:-4]
FijB5 = FijB4[:-4,:-4]

# get sigmas for tau, w, mnu, biases held fixed
sigmasZ5 = getSigmas(FijZ5)
sigmasB5 = getSigmas(FijB5)

print paramList5

['ombh2' 'omch2' 'cosmomc_theta' 'As' 'ns']


In [172]:
# excise biases and remake sigmas
paramList6 = paramList[:-4]
FijZ6 = FijZ[:-4,:-4]
FijB6 = FijBp[:-4,:-4]

# get sigmas for tau, w, mnu, biases held fixed
sigmasZ6 = getSigmas(FijZ6)
sigmasB6 = getSigmas(FijB6)

print paramList6

['ombh2', 'omch2', 'cosmomc_theta', 'As', 'ns', 'tau', 'mnu', 'w']


# Make a table of results

In [173]:
from prettytable import PrettyTable

print 'with no parameters fixed: '
t = PrettyTable(['Parameter', 'sigma_Z(k,g)','sigma_B(k,g)','percent diff.'])
for i,p in enumerate(paramList):
    Zi = sigmasZ[i]
    Bi = sigmasB[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'with w, tau fixed: '
t = PrettyTable(['Parameter', 'sigma_Z(k,g)','sigma_B(k,g)','percent diff.'])
for i,p in enumerate(paramList3):
    Zi = sigmasZ3[i]
    Bi = sigmasB3[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'with w, tau, mnu fixed: '
t = PrettyTable(['Parameter', 'sigma_Z(k,g)','sigma_B(k,g)','percent diff.'])
for i,p in enumerate(paramList4):
    Zi = sigmasZ4[i]
    Bi = sigmasB4[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'with w, tau, mnu, biases fixed: '
t = PrettyTable(['Parameter', 'sigma_Z(k,g)','sigma_B(k,g)','percent diff.'])
for i,p in enumerate(paramList5):
    Zi = sigmasZ5[i]
    Bi = sigmasB5[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t

print 'with biases fixed: '
t = PrettyTable(['Parameter', 'sigma_Z(k,g)','sigma_B(k,g)','percent diff.'])
for i,p in enumerate(paramList6):
    Zi = sigmasZ6[i]
    Bi = sigmasB6[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi, "%5.3g" % diff ] )
print t



with no parameters fixed: 
+---------------+--------------+--------------+---------------+
|   Parameter   | sigma_Z(k,g) | sigma_B(k,g) | percent diff. |
+---------------+--------------+--------------+---------------+
|     ombh2     |   0.00231    |   0.00236    |      1.84     |
|     omch2     |   0.00697    |   0.00705    |       1.1     |
| cosmomc_theta |   8.94e-05   |   7.8e-05    |     -12.7     |
|       As      |   1.35e-10   |   1.39e-10   |      2.69     |
|       ns      |    0.0194    |    0.0196    |      1.08     |
|      tau      |     7.06     |     7.12     |     0.807     |
|      mnu      |    0.0964    |    0.101     |      5.22     |
|       w       |    0.0412    |    0.0405    |      -1.9     |
|      bin1     |    0.0103    |    0.0105    |      1.62     |
|      bin2     |    0.0125    |    0.0131    |      4.95     |
|      bin3     |    0.0162    |    0.0164    |      1.01     |
|      bin4     |    0.0179    |    0.0192    |      6.74     |
+------------

In [ ]:
# notes on LSST dndz - move this somewhere more useful
def z0_of_i(i):
    print i
    return 0.0417 * i − 0.744
print z0_of_i(25)

# Collected parameter errors from various emails

In [174]:
# Byeonghee's results using nonlinear power spectra
"""sigma(b)/b (4 bins) =  [ 0.0063022   0.00560226  0.00540911  0.00466538]
sigma(cosmomc_theta) =  7.58276477127e-05
sigma(\Omega_bh^2) =  0.00206850078114
sigma(\Omega_ch^2) =  0.00624934014317
sigma(n_s) =  0.0190489068958
sigma(10^9A_s) =  0.129011144081
sigma(m_{nu}) =  0.11056107791
sigma(w) =  0.0411035064527
sigma(tau) =  8.25658593413"""
sigmasB4 = {
    'cosmomc_theta' : 7.58276477127e-05,
    'ombh2' : 0.00206850078114,
    'omch2' : 0.00624934014317,
    'ns' : 0.0190489068958,
    'As' : 0.129011144081e-9,
    'mnu' : 0.11056107791,
    'w' : 0.0411035064527,
    'tau' : 8.25658593413,
    'bin1' : 0.0063022,
    'bin2' : 0.00560226,
    'bin3' : 0.00540911,
    'bin4' : 0.00466538
}
"""Now do the one that has had w and tau removed
sigma(b) (4 bins) =  [ 0.00553446  0.00386324  0.00346112  0.0028168 ]
sigma(cosmomc_theta) =  5.3458847331e-05
sigma(\Omega_bh^2) =  0.00164438257858
sigma(\Omega_ch^2) =  0.00435030886836
sigma(n_s) =  0.00865043065069
sigma(10^9A_s) =  0.0723781128272
sigma(m_{nu}) =  0.0667167394008
"""
sigmasB5_no_w_tau = {
    'cosmomc_theta' : 5.3458847331e-05,
    'ombh2' : 0.00164438257858,
    'omch2' : 0.00435030886836,
    'ns' : 0.00865043065069,
    'As' : 0.0723781128272e-9,
    'mnu' : 0.0667167394008,
    'bin1' : 0.00553446,
    'bin2' : 0.00386324,
    'bin3' : 0.00346112,
    'bin4' : 0.0028168
}

sigmasB = sigmasB4
#sigmasB = sigmasB5_no_w_tau


In [175]:
# more revised results from Byeonghee; these for linear power spectra
"""
+---------------+--------------+--------------+
|   Parameter   | sigma_Z(k,g) | sigma_B(k,g) | 
+---------------+--------------+--------------+
|     ombh2     |   0.00135    |   0.00103    | 
|     omch2     |   0.00313    |   0.00201    |  
| cosmomc_theta |   3.52e-05   |   2.50e-05   |   
|       As      |   4.45e-11   |   3.11e-11   |     
|       ns      |   0.00564    |   0.00330    |     
|      mnu      |    0.0187    |    0.0273    |      
+---------------+--------------+--------------+

+---------------+--------------+
|   Parameter   | sigma_B(k,g) | 
+---------------+--------------+
|     ombh2     |   0.00103    | 
|     omch2     |   0.00200    |  
| cosmomc_theta |   2.13e-05   |  
|       As      |   2.50e-11   |    
|       ns      |   0.00284    |      
+---------------+--------------+
"""
sigmasB6 = [0.00103,0.00201,2.50e-05,3.11e-11,0.00330,0.0273]
sigmasB7 = [0.00103,0.00200,2.13e-05,2.50e-11,0.00284]

